In [31]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, LeaveOneOut
from scipy import stats as st
import matplotlib.pyplot as plt


# Chargement des données

In [5]:
df= pd.read_csv('https://www.labri.fr/perso/zemmari/datasets/cars_data-class.csv', delimiter =',')
df.head()

,Taille_Auto,Poids_Auto,Acceleration,Annee_Fabrication,Kilometrage,Nombre_Portes,Vitesse_Max,Prix_Assurance,Categorie_Voiture,Decote,Impact_Environnement
0,3.790317,8869.946374,2.038688,2006,135542.067972,5,195.289066,184.125577,1,0.104588,1
1,3.375892,1126.174079,2.067411,2010,33710.551662,5,249.979656,858.935088,3,0.346064,1
2,4.844971,6100.019066,4.839676,2016,112977.279109,5,235.763320,302.138838,4,0.394582,3
3,2.316172,5771.509867,1.202574,2014,50139.727635,4,216.802020,1298.671966,5,0.422048,2
4,2.618825,5676.666211,5.470806,2009,191188.750210,4,234.004505,702.626517,2,0.279959,1


In [9]:
print('Taille de l\'échantillon : ',len(df))
print('Différentes classes d\'impact sur l\'environnement : ', df.Impact_Environnement.unique())

Taille de l'échantillon :  1000
Différentes classes d'impact sur l'environnement :  [1 3 2 0]


In [46]:
size0 = len(df[df.Impact_Environnement==0])
size1 = len(df[df.Impact_Environnement==1])
size2 = len(df[df.Impact_Environnement==2])
size3 = len(df[df.Impact_Environnement==3])
print(size0, size1, size2, size3)
np.unique(df.Impact_Environnement, return_counts=True)

36 537 42 385


(array([0, 1, 2, 3]), array([ 36, 537,  42, 385]))

In [40]:
X = df.drop(['Impact_Environnement'], axis=1)
y = df['Impact_Environnement']
print(y)

0      1
1      1
2      3
3      2
4      1
      ..
995    3
996    1
997    1
998    1
999    0
Name: Impact_Environnement, Length: 1000, dtype: int64


In [36]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [37]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [25]:
from imblearn.over_sampling import SMOTE
smote = SMOTE()
X_train_oversample, y_train_oversample = smote.fit_resample(X_train, y_train)
print(len(X_train_oversample), len(X_train))
print(len(y_train_oversample), len(y_train))

ModuleNotFoundError: No module named 'imblearn'

In [53]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report, balanced_accuracy_score, confusion_matrix
from sklearn.model_selection import GridSearchCV

param_grid = {
    'C': [0.1, 1, 10, 100],  # Différentes valeurs pour le paramètre de régularisation C
    'kernel': ['linear', 'rbf']  # Différents types de noyau
}

# Initialiser le modèle SVM
svm = SVC()

grid_search = GridSearchCV(estimator=svm, param_grid=param_grid, cv=4, scoring='accuracy')

# Adapter le modèle avec les données d'entraînement
grid_search.fit(X_train, y_train)

# Obtenir les meilleurs paramètres
best_params = grid_search.best_params_
print("Meilleurs paramètres:", best_params)


best_svm = SVC(**best_params)
best_svm.fit(X_train, y_train)
predictions = best_svm.predict(X_test)

# Évaluer les performances du modèle
from sklearn.metrics import classification_report, confusion_matrix
print(balanced_accuracy_score(y_test, predictions))
print(classification_report(y_test, predictions))
print(confusion_matrix(y_test, predictions))

clf_SVC = SVC()
clf_SVC.fit(X_train, y_train)
y_pred_SVC = clf_SVC.predict(X_test)
print(classification_report(y_test, y_pred_SVC))
print(balanced_accuracy_score(y_test, y_pred_SVC))
print(confusion_matrix(y_test, y_pred_SVC))

Meilleurs paramètres: {'C': 0.1, 'kernel': 'linear'}
0.45264968614196044
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         8
           1       0.79      0.98      0.88       109
           2       0.00      0.00      0.00         7
           3       0.97      0.83      0.89        76

    accuracy                           0.85       200
   macro avg       0.44      0.45      0.44       200
weighted avg       0.80      0.85      0.82       200

[[  0   8   0   0]
 [  0 107   0   2]
 [  0   7   0   0]
 [  0  13   0  63]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         8
           1       0.77      0.97      0.86       109
           2       0.00      0.00      0.00         7
           3       0.95      0.79      0.86        76

    accuracy                           0.83       200
   macro avg       0.43      0.44      0.43       200
weighted avg       0.78      0.83  

/usr/lib/python3/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/lib/python3/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/lib/python3/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/lib/python3/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarn

In [45]:

loo = LeaveOneOut()
results = np.zeros(len(y)) #y_train_oversample
for train, test in loo.split(X):

    clf = SVC()
    clf.fit(X.iloc[train], y.iloc[train])
    print(clf.predict(X.iloc[test]))
    results[test] = clf.predict(X.iloc[test])
print(classification_report(y, results))
print(balanced_accuracy_score(y, results))
print(confusion_matrix(y, results))

[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]


/usr/lib/python3/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/lib/python3/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/lib/python3/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
